In [55]:
!pip install janome

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%pwd

'/content'

# 自然言語処理入門
* 自然言語処理を体験する
* 簡単な文書の分析ができるようになる

In [0]:
import pandas as pd
import numpy as np
import re
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from nltk import word_tokenize, FreqDist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, GRU, Embedding, Input
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils, to_categorical
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from tensorflow.python.keras import models

# 【問題1】BoWとN-gram(手計算)
目的

古典的かつ強力な手法BoWとN-gramの理解
以下は俳優K.Kさんのつぶやき(コーパス)です。

文1: 今撮影中で〜す！  
文2: 今休憩中で〜す(^^)  
文3: 今日ドラマ撮影で〜す！  
文4: 今日、映画瞬公開で〜す！！！  

【問】  
特殊文字除去(!や〜など)、単語分割をし以下の2パターンで文1〜文4を数値化(ベクトル化)してください。  

BoW(1-gram)  
BoW(2-gram)  
手計算の後見やすい形にしてください。

In [4]:
import pandas as pd

vocabulary = ["I", "love", "this", "is", "the", "baseball"]
ms_kk_texts = ["I love baseball !!", "I love this !"]
texts_vec = [[1,1,0,0,0,1], [1,1,1,0,0,0]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram

,I,love,this,is,the,baseball
I love baseball !!,1,1,0,0,0,1
I love this !,1,1,1,0,0,0


＜回答＞

In [5]:
import pandas as pd

vocabulary = ["今", "撮影","中", "です", "休憩", "今日", "ドラマ", "撮影","映画","公開"]
ms_kk_texts = ["今撮影中で〜す！", "今休憩中で〜す(^^)","今日ドラマ撮影で〜す！","今日、映画瞬公開で〜す！！！"]
texts_vec = [[1,1,1,1,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0], [0,0,0,0,0,1,1,1,1,0], [0,0,0,1,0,1,0,0,1,1,]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram

,今,撮影,中,です,休憩,今日,ドラマ,撮影,映画,公開
今撮影中で〜す！,1,1,1,1,0,0,0,0,0,0
今休憩中で〜す(^^),1,0,1,1,1,0,0,0,0,0
今日ドラマ撮影で〜す！,0,0,0,0,0,1,1,1,1,0
今日、映画瞬公開で〜す！！！,0,0,0,1,0,1,0,0,1,1


# 【問題2】TF-IDF(手計算)
目的

* 古典的かつ強力なTF-IDFの理解
* 標準的なTF-IDFの公式   

Term Frequency:
<center>$tf(t,d) = \frac{n_{t,d}}{\sum_{s \in d}n_{s,d}}$</center>  
Inverse Document Frequency:  
<center>$idf(t) = \log_2{\frac{N}{df(t)}}$</center>  
TF-IDF:  
<center>$tfidf(t, d) = tf(t, d) \times idf(t)$</center>  

## 【問】  
問題1のコーパスを使って、文1〜文4をTFIDFで数値化(ベクトル化)してください。   
問題1と同様、手計算の後見やすい形にしてください。  

正解例  
tfidf(今, 文書1) = 0.25 になります。

In [6]:
#同じ行名・列名のデータフレームを作成
import numpy as np
df_tf = pd.DataFrame(columns=df_bow_1gram.columns,index =df_bow_1gram.index)

#ｔｆの計算：各文の単語数を計算した後、各単語の出現割合を計算
for i in range(0,df_tf.shape[0]):
    n_t_d = sum(df_bow_1gram.iloc[i,:])
    for j in range(0,df_tf.shape[1]):
        df_tf.iloc[i,j]=df_bow_1gram.iloc[i,j]/n_t_d
display(df_tf)

,今,撮影,中,です,休憩,今日,ドラマ,撮影,映画,公開
今撮影中で〜す！,0.25,0.25,0.25,0.25,0,0,0,0,0,0
今休憩中で〜す(^^),0.25,0,0.25,0.25,0.25,0,0,0,0,0
今日ドラマ撮影で〜す！,0,0,0,0,0,0.25,0.25,0.25,0.25,0
今日、映画瞬公開で〜す！！！,0,0,0,0.25,0,0.25,0,0,0.25,0.25


In [7]:
#idtの計算：全文章数からその単語が出現する文章の数を割って、その単語が出現した場合の特徴性・固有性を判断
#同じ列名のデータフレームを作成
df_idt = pd.DataFrame(columns=df_bow_1gram.columns,index=['df(t)','idf(t)']) #出現する文章数をカウントしていく
df_idt = df_idt.fillna(0)
display("初期状態",df_idt)

#各単語要素のdf(t)を計算
N = df_bow_1gram.shape[0]

for i in range(df_bow_1gram.shape[0]) :
    for j in range(df_bow_1gram.shape[1]):
        if df_bow_1gram.iloc[i,j] > 0:
            df_idt.iloc[0,j] += 1
            
display("df(t)計算処理後",df_idt)

#各単語要素のidf(t)を計算
df_idt.loc['idf(t)'] = np.log2(N / df_idt.loc['df(t)'])
display("idf(t)計算処理後",df_idt)

'初期状態'

,今,撮影,中,です,休憩,今日,ドラマ,撮影,映画,公開
df(t),0,0,0,0,0,0,0,0,0,0
idf(t),0,0,0,0,0,0,0,0,0,0


'df(t)計算処理後'

,今,撮影,中,です,休憩,今日,ドラマ,撮影,映画,公開
df(t),2,1,2,3,1,2,1,1,2,1
idf(t),0,0,0,0,0,0,0,0,0,0


'idf(t)計算処理後'

,今,撮影,中,です,休憩,今日,ドラマ,撮影,映画,公開
df(t),2.0,1.0,2.0,3.000000,1.0,2.0,1.0,1.0,2.0,1.0
idf(t),1.0,2.0,1.0,0.415037,2.0,1.0,2.0,2.0,1.0,2.0


In [8]:
#各文章×単語要素のtf-idf(t,d)を計算
df_tf_idf = pd.DataFrame(columns=df_bow_1gram.columns,index =df_bow_1gram.index)
df_tf_idf = df_tf * df_idt.loc['idf(t)']
display(df_tf_idf)
#df_idt.loc['tf-idf(t,d)'] = df_idt.loc['df(t)'] * df_idt.loc['idf(t)']
#display("tf-idf(t)計算処理後",df_idt)

,今,撮影,中,です,休憩,今日,ドラマ,撮影,映画,公開
今撮影中で〜す！,0.25,0.5,0.25,0.103759,0,0,0,0,0,0
今休憩中で〜す(^^),0.25,0,0.25,0.103759,0.5,0,0,0,0,0
今日ドラマ撮影で〜す！,0,0,0,0,0,0.25,0.5,0.5,0.25,0
今日、映画瞬公開で〜す！！！,0,0,0,0.103759,0,0.25,0,0,0.25,0.5


# 【問題3】テキストクリーニング(プログラミング)
目的

実データ対応のためのテキストクリーニングの理解
正規表現の理解
実際のテキストデータは非常に汚いことが多いです。   
以下は3/6(水)にnoroさんがSlackで発言した文章で、良い例です。

### 【問】
このテキストに以下の処理を施してください。  


<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:

  
  
* urlを削除
* 【〇〇】を削除
* 改行等の特殊文字を削除
* 絵文字除去  


ここではしませんが、数字を文字列NUMBERに置き換える処理をよくします。

In [9]:
# 正規表現操作のライブラリ
import re
# 対象テキストデータ
text = "<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:"
display("初期テキスト",text)
# re.compileを使うと処理が早くなります
#BAD_SYMBOL = re.compile('[\n*！`]+') #基本的な文字の削除
# re.sub(r'[\n*！`]+', '', text)でもできます
text = re.sub(r'[\n*！`]+', '', text)
text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text) #URLの削除 
text = re.sub('<.*?>', "", text) #<>で入っているHTMLタグを削除
text = re.sub('【.*?】', "", text) #【から始まって】で終わる２文字以上の文字列
text = re.sub(':[^0-9]+.*?:', "", text) #：(非数字):の部分の削除
clean_text = text
display("最終テキスト",text)

'初期テキスト'

'<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:'

'最終テキスト'

' 有償のRubyMineやPyCharmの 6ヶ月間100%OFFクーポン をご希望者の方先着100名様に贈呈いたしますこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、 2019年3月20日（水）22:00まで とさせていただきます。ふるってのご希望をお待ちしております '

### ※HTMLを正しく表現した形に直し、取り除く例

In [10]:
from bs4 import BeautifulSoup
text = "<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:"
changed_text = BeautifulSoup(text)
changed_changed_text = changed_text.get_text()
display("htmlを正しく配置した文章",changed_text)
display("htmlを除いた文章",changed_changed_text)


'htmlを正しく配置した文章'

<html><body><p> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*

この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。

```
RubyMine
<https:></https:>

PyCharm
<https:></https:>
```

「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。

期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。
ふるってのご希望をお待ちしております！ :smile:</p></body></html>

'htmlを除いた文章'

' *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n\n\nPyCharm\n\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:'

### Tips NLPのLinuxコマンド  
これまでpythonでファイルを読み込んで処理をしていましたが、  
簡単な作業においてはlinuxコマンドの方がメモリの使用料が半分以下だったりとパフォーマンスが良いです。  

例えばファイルの行数を数えたい場合、pythonでわざわざ書くのは面倒です。  
以下の1行のコマンドで実行できます。  

wc -l 〇〇.txt  
また分割したい場合はsplit  
並び替えたい場合はsort  
置換にはsed  
文の先頭、後頭部分を見たければhead,tail  
など便利なコマンドがあります。  
詳しく知りたい方はNLP100本ノックで調べてみてください。

# 【問題4】形態素解析
目的

形態素解析の理解  
形態素解析のツールはMecabやJanomeなど様々ですが、  
ここでは手軽に導入できるJanomeを使います。  
Janome document  

【問】
上記のクリーニングしたテキストをJanomeを用いて形態素解析をし、  
名詞または動詞の単語を抜き出してください。  

In [11]:
from janome.tokenizer import Tokenizer
tokenizer = Tokenizer()
[token for token in tokenizer.tokenize(clean_text, wakati=True)]

['有償',
 'の',
 'RubyMine',
 'や',
 'PyCharm',
 'の',
 ' ',
 '6',
 'ヶ月',
 '間',
 '100',
 '%',
 'OFF',
 'クーポン',
 ' ',
 'を',
 'ご',
 '希望',
 '者',
 'の',
 '方',
 '先着',
 '100',
 '名',
 '様',
 'に',
 '贈呈',
 'いたし',
 'ます',
 'この',
 '度',
 '、',
 'RubyMine',
 'や',
 'PyCharm',
 'の',
 'メーカー',
 'で',
 'ある',
 'JetBrains',
 '社',
 'へ',
 'の',
 'クーポン',
 'コード',
 'の',
 '提供',
 '交渉',
 'が',
 '実り',
 '、',
 '100',
 'クーポン',
 'を',
 'いただく',
 'こと',
 'が',
 'でき',
 'まし',
 'た',
 '。',
 'RubyMinePyCharm',
 '「',
 'ご',
 '希望',
 'の',
 '方',
 'は',
 '、',
 '手',
 'を',
 '挙げ',
 'て',
 '」',
 '方式',
 'で',
 '、',
 'ご',
 '希望',
 'の',
 '方',
 'は',
 'この',
 '投稿',
 'の',
 '手',
 'あげ',
 'スタンプ',
 'を',
 'クリック',
 'し',
 'て',
 'ください',
 '。',
 '期限',
 'は',
 '、',
 ' ',
 '2019',
 '年',
 '3',
 '月',
 '20',
 '日',
 '（',
 '水',
 '）',
 '22',
 ':',
 '00',
 'まで',
 ' ',
 'と',
 'さ',
 'せ',
 'て',
 'いただき',
 'ます',
 '。',
 'ふるって',
 'の',
 'ご',
 '希望',
 'を',
 'お待ち',
 'し',
 'て',
 'おり',
 'ます']

In [12]:
word_list=[]
 
t = Tokenizer()
tokens = t.tokenize(clean_text,) 
 
for token in tokens:
    word = token.surface
    partOfSpeech = token.part_of_speech.split(',')[0]
    partOfSpeech2 = token.part_of_speech.split(',')[1]
     
    if (partOfSpeech == "名詞")or(partOfSpeech =="動詞"):
        if (partOfSpeech2 != "非自立") and (partOfSpeech2 != "代名詞") and (partOfSpeech2 != "数"):
            word_list.append(word)
 
word_list

['有償',
 'RubyMine',
 'PyCharm',
 'ヶ月',
 '間',
 '%',
 'OFF',
 'クーポン',
 '希望',
 '者',
 '先着',
 '名',
 '様',
 '贈呈',
 'RubyMine',
 'PyCharm',
 'メーカー',
 'JetBrains',
 '社',
 'クーポン',
 'コード',
 '提供',
 '交渉',
 '実り',
 'クーポン',
 'いただく',
 'でき',
 'RubyMinePyCharm',
 '希望',
 '手',
 '挙げ',
 '方式',
 '希望',
 '投稿',
 '手',
 'あげ',
 'スタンプ',
 'クリック',
 'し',
 '期限',
 '年',
 '月',
 '日',
 '水',
 ':',
 'さ',
 'せ',
 '希望',
 'お待ち',
 'し']

# 【問題5】ニュースの分析
目的
日本語の自然言語処理の体験
類似度の理解

In [0]:
#ダウンロード
#!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

In [0]:
#解凍
#!tar zxf ldcc-20140209.tar.gz

In [15]:
!cat text/README.txt

livedoor ニュースコーパス


概要
-----------------
本コーパスは、NHN Japan株式会社が運営する「livedoor ニュース」のうち、下記のクリエイティブ・コモンズ
ライセンスが適用されるニュース記事を収集し、可能な限りHTMLタグを取り除いて作成したものです。

- トピックニュース
  http://news.livedoor.com/category/vender/news/
- Sports Watch
  http://news.livedoor.com/category/vender/208/
- ITライフハック
  http://news.livedoor.com/category/vender/223/
- 家電チャンネル
  http://news.livedoor.com/category/vender/kadench/
- MOVIE ENTER
  http://news.livedoor.com/category/vender/movie_enter/
- 独女通信
  http://news.livedoor.com/category/vender/90/
- エスマックス
  http://news.livedoor.com/category/vender/smax/
- livedoor HOMME
  http://news.livedoor.com/category/vender/homme/
- Peachy
  http://news.livedoor.com/category/vender/ldgirls/

収集時期：2012年9月上旬


ライセンス
-----------------
各記事ファイルにはクリエイティブ・コモンズライセンス「表示 - 改変禁止」
（http://creativecommons.org/licenses/by-nd/2.1/jp/）が適用されます。
クレジット表示についてはニュースカテゴリにより異なるため、サブディレクトリにある
それぞれの LICENSE.txt をご覧ください。

livedoor はNHN Japan株式会社の登録商標です。


記事ファイルのフォーマット
-----------------
記事ファイルは以下のフォーマットにしたが

In [0]:
# サブフォルダまで賢く読み込んでもらう
from sklearn.datasets import load_files
# encodingをutf-8指定して読み込み
bin_data = load_files('./text', encoding='utf-8')
documents = bin_data.data
# 今回はラベルが無いと仮定してください
# targets = bin_data.target

# 【問】
以下の流れでニュースを分析してください。  
まずどんなニュースなのか読んでみる  
出現単語をカウントして分析する  
テキストをクリーニングする  
BoW + TFIDFでベクトル化する  
あるニュースに一番cos類似度が近いニュースを出力する関数の作成  
別の類似度手法を1つ調べて上の関数に組み込む(切り替えられるようにする)  
なぜそのような結果になったのか考察する  

In [114]:
print("元の記事数:",len(documents))

#メモリの都合上、記事数を500に減らす
documents = documents[:500]

print("メモリ上限により減らした記事数:",len(documents))

元の記事数: 7376
減らした記事数: 500


### ニュースを読んでみる

In [31]:
documents[9]

'http://news.livedoor.com/article/detail/6798260/\n2012-07-27T17:00:00+0900\n音声認識で英語の発音練習しよう Siriを使うiPhoneアプリReal英会話【デジ通】\nアップルのSiriは音声認識機能を持つバーチャルアシスタントだ。この音声認識機能を使用して、英会話の発音練習に活用している方もいると思われるが、英会話学習アプリでもこの機能を活用し始めている。その1つが、LT Boxの「Real英会話」だ。Real英会話はiPhoneやiPadなどに対応したユニバーサルアプリで、以前から実生活で使われるようなリアルな英会話の学習ができた人気のアプリだ。2012年5月に行われたアップデート（バージョン3.1）では、Siriによる音声認識を使った発音練習機能が追加されている。\n\nReal英会話自体は、TOEICなどの練習よりも、主に日常生活で使われているような英会話のリスニングなどを練習できるアプリで、それ自体にも価値があるが、発音認識機能がついたことでさらに価値を増している。\n\n単語や文法などは独学で学習できたとしても、発音練習を独学でやるのは難しい。最近利用が本格化してきた音声認識機能を活用すれば、独学での発音練習も現実的になってきた。\n\n■Siriの音声認識機能をそのまま利用する\n実際に試してみると、アプリで学習する日常会話をSiriによる音声認識機能を使ってそのまま学習できるようになっている。アプリのフレーズクイズ内に発音練習があり、ここで練習できる。\n\n発音練習画面では、日本語の意味と、英語が表示される。ここに表示される英語を発音し音声認識で正しく認識できれば、そのフレーズを学習したことになる。\n\n認識精度はSiriの音声認識機能をそのまま使用するため非常に高い。しかし、日本の多くの学習者は日本語アクセントの英語で発音することが多いため、認識という点ではかなり厳しいのではないかと思われる。\n\nSiriの音声認識機能は、英語でも、アメリカ、イギリス、オーストラリアの3つの地域別に設定できる。日本語アクセントの英語という設定はないので、Real英会話アプリで機械的に緩く認識するようなことはない。日本語アクセントの英語で認識させようと思っても、ネイティブが聞き間違え

音声認識機能を用いた英会話学習とそのシステムについてのニュース。

### 出現単語をカウント

In [33]:
from janome.analyzer import Analyzer

a = Analyzer(token_filters=[TokenCountFilter()])

g_count = list(a.analyze(documents[9]))

print(g_count)

[('http', 1), ('://', 1), ('news', 1), ('.', 5), ('livedoor', 1), ('com', 1), ('/', 5), ('article', 1), ('detail', 1), ('6798260', 1), ('\n', 12), ('2012', 2), ('-', 3), ('07', 1), ('27', 1), ('T', 1), ('17', 1), (':', 2), ('00', 2), ('+', 2), ('0900', 1), ('音声', 13), ('認識', 23), ('で', 25), ('英語', 9), ('の', 40), ('発音', 17), ('練習', 12), ('しよ', 1), ('う', 3), (' ', 9), ('Siri', 7), ('を', 23), ('使う', 1), ('iPhone', 4), ('アプリ', 8), ('Real', 7), ('英会話', 12), ('【', 2), ('デジ', 3), ('通', 6), ('】', 2), ('アップル', 1), ('は', 23), ('機能', 12), ('持つ', 1), ('バーチャル', 1), ('アシスタント', 1), ('だ', 6), ('。', 23), ('この', 2), ('使用', 2), ('し', 9), ('て', 17), ('、', 32), ('に', 22), ('活用', 4), ('いる', 7), ('方', 1), ('も', 11), ('と', 5), ('思わ', 2), ('れる', 7), ('が', 22), ('学習', 8), ('始め', 1), ('その', 2), ('1', 3), ('つ', 3), ('LT', 1), ('Box', 1), ('「', 3), ('」', 3), ('や', 2), ('iPad', 2), ('など', 4), ('対応', 1), ('た', 10), ('ユニバーサルアプリ', 1), ('以前', 1), ('から', 1), ('実生活', 1), ('使わ', 2), ('よう', 5), ('な', 5), ('リアル', 1), ('でき',

### テキストをクリーニングする

In [34]:
#<>
kakko1 = '<.*?>'

#【】
kakko2 = '【.*?】'

#特殊文字
tokusyu = '[\n\*`\s■◆○★●]'

#URL
url = '(http.*?\n)'

#日時
date = '(\d{4}-.*?\+\d{4})'

#連結
reg_str = f'{kakko1}|{kakko2}|{tokusyu}|{url}|{date}'

#テキストクリーニング
document = []

for i in documents:
    remove = re.compile(reg_str)
    document.append(re.sub(remove, '', i))

print(document[9])

音声認識で英語の発音練習しようSiriを使うiPhoneアプリReal英会話アップルのSiriは音声認識機能を持つバーチャルアシスタントだ。この音声認識機能を使用して、英会話の発音練習に活用している方もいると思われるが、英会話学習アプリでもこの機能を活用し始めている。その1つが、LTBoxの「Real英会話」だ。Real英会話はiPhoneやiPadなどに対応したユニバーサルアプリで、以前から実生活で使われるようなリアルな英会話の学習ができた人気のアプリだ。2012年5月に行われたアップデート（バージョン3.1）では、Siriによる音声認識を使った発音練習機能が追加されている。Real英会話自体は、TOEICなどの練習よりも、主に日常生活で使われているような英会話のリスニングなどを練習できるアプリで、それ自体にも価値があるが、発音認識機能がついたことでさらに価値を増している。単語や文法などは独学で学習できたとしても、発音練習を独学でやるのは難しい。最近利用が本格化してきた音声認識機能を活用すれば、独学での発音練習も現実的になってきた。Siriの音声認識機能をそのまま利用する実際に試してみると、アプリで学習する日常会話をSiriによる音声認識機能を使ってそのまま学習できるようになっている。アプリのフレーズクイズ内に発音練習があり、ここで練習できる。発音練習画面では、日本語の意味と、英語が表示される。ここに表示される英語を発音し音声認識で正しく認識できれば、そのフレーズを学習したことになる。認識精度はSiriの音声認識機能をそのまま使用するため非常に高い。しかし、日本の多くの学習者は日本語アクセントの英語で発音することが多いため、認識という点ではかなり厳しいのではないかと思われる。Siriの音声認識機能は、英語でも、アメリカ、イギリス、オーストラリアの3つの地域別に設定できる。日本語アクセントの英語という設定はないので、Real英会話アプリで機械的に緩く認識するようなことはない。日本語アクセントの英語で認識させようと思っても、ネイティブが聞き間違えるように間違えて認識されてしまう。Real英会話での学習のコツとしては、発音例も画面をタップすればすぐに再生されるので、英文を読むのではなく、発音例をまねてそのまま発音することだ。筆者も日本語アクセントの英語がやっとだが、なか

In [35]:
#形態素解析(複合名詞　+ 動詞を抽出)
corpus = []

a = Analyzer(token_filters=[CompoundNounFilter()])
for text in tqdm_notebook(document):
    corpus.append(' '.join([token.surface for token in a.analyze(text) 
                                    if token.part_of_speech.split(',')[0] in ['動詞', '名詞']]))

In [36]:
print(corpus[9])

音声認識 英語 発音練習 しよ Siri 使う iPhoneアプリReal英会話アップル Siri 音声認識機能 持つ バーチャルアシスタント 音声認識機能 使用 し 英会話 発音練習 活用 し いる 方 いる 思わ れる 英会話学習アプリ 機能 活用 し 始め いる 1 LTBox Real英会話 Real英会話 iPhone iPad 対応 し ユニバーサルアプリ 以前 実生活 使わ れる よう リアル 英会話 学習 でき 人気 アプリ 2012年5月 行わ れ アップデート バージョン3.1 Siri 音声認識 使っ 発音練習機能 追加 さ れ いる Real英会話自体 TOEIC 練習 主 日常生活 使わ れ いる よう 英会話 リスニング 練習 できる アプリ それ自体 価値 ある 発音認識機能 つい こと 価値 増し いる 単語 文法 独学 学習 でき 発音練習 独学 やる の 最近利用 本格化 し き 音声認識機能 活用 すれ 独学 発音練習 現実的 なっ き Siri 音声認識機能 利用 する 試し みる アプリ 学習 する 日常会話 Siri 音声認識機能 使っ 学習 できる よう なっ いる アプリ フレーズクイズ内 発音練習 あり ここ 練習 できる 発音練習画面 日本語 意味 英語 表示 さ れる ここ 表示 さ れる 英語 発音 し 音声認識 認識 できれ フレーズ 学習 し こと なる 認識精度 Siri 音声認識機能 使用 する ため非常 日本 多く 学習者 日本語アクセント 英語 発音 する こと ため 認識 点 の 思わ れる Siri 音声認識機能 英語 アメリカ イギリス オーストラリア 3 地域別 設定 できる 日本語アクセント 英語 設定 Real英会話アプリ 機械的 認識 する よう こと 日本語アクセント 英語 認識 さ せよ 思っ ネイティブ 聞き 間違える よう 間違え 認識 さ れ しまう Real英会話 学習 コツ 発音例 画面 タップ すれ 再生 さ れる 英文 読む の 発音例 まね 発音 する こと 筆者 日本語アクセント 英語 認識 さ れ フレーズ まね し 発音 する こと 認識 さ れる こと 発音 多く 日本人 認識精度 発音練習 1 活用 する もってこい アプリ Real英会話上倉賢@kami

### BoW + TFIDFでベクトル化する

In [38]:
#BoW
count = CountVectorizer()

docs = np.array(corpus)
bag = count.fit_transform(docs)
print("BoW",bag.toarray())

#TF-IDF
tfidf = TfidfTransformer()
vector = tfidf.fit_transform(bag).toarray()
print("TF-IDF",vector)

BoW [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
TF-IDF [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### あるニュースにcos類似度が最も近いニュースを出力する関数を作成

In [0]:
def similar(documents, news_num=1, sim='cos'):
    '''
    cos類似度を求める
    
    Parameters
    --------------
    documents : リスト
        様々なニュース
    news_num : int
        何番目のニュースか(デフォルト : 1(1番目のニュース))
    sim : str
        類似度(デフォルト : cos(cos類似度))
    ''' 
    #様々なニュース
    #形態素解析してリストに戻す(複合名詞　+ 動詞)
    corpus = []
    a = Analyzer(token_filters=[CompoundNounFilter()])
    for document in tqdm_notebook(documents):
        corpus.append(' '.join([token.surface for token in a.analyze(document) 
                                        if token.part_of_speech.split(',')[0] in ['動詞', '名詞']]))
    
    #Bow
    count = CountVectorizer()
    docs = np.array(corpus)
    bag = count.fit_transform(docs)
    
    #TF-IDF
    fidf = TfidfTransformer()
    vector = tfidf.fit_transform(bag).toarray()
    
    #ターゲットのニュース
    news_num -= 10 #ニュースの番号
    news = docs[news_num] #ニュースの内容
    news_corpus = corpus[news_num] #形態素解析したニュース
    news_vector = vector[news_num] #TF-IDFしたニュース
    

    #cos類似度の場合
    if sim == 'cos':
        
        #初期値
        cos_max = 0 #最大のcos類似度
        index = 0 #cos類似度が最大の時のindex

        #cos類似度を計算
        for i, doc_vec in enumerate(vector):        
            cos = np.dot(news_vector, doc_vec) / (np.linalg.norm(news_vector) * np.linalg.norm(doc_vec))

            #同じ文章ならpass
            if int(cos) == 1:
                pass

            #cos類似度が最高ならcos類似度とインデックスを更新
            else:
                if cos_max < cos:
                    cos_max = cos
                    index = i

        print('cos類似度　:　{:.3}'.format(cos_max) + '\n' + documents[index])
        
        
    #doc2vec
    elif sim == 'doc2vec':
        
        #初期値
        training_docs = [] 

        #ターゲットのニュースをリストに入れる
        sent1 = TaggedDocument(words=news_corpus.split(' '), tags=[news])
        training_docs.append(sent1)
        
        #全てのニュースをリストに入れる
        for i, (doc_corpus, doc) in enumerate(zip(corpus, documents)):
            
            #ターゲットのニュースと同じニュースはpass、それ以外はリストへ
            if i == news_num:
                pass
            
            else:
                sent2 = TaggedDocument(words=doc_corpus.split(' '), tags=[doc])
                training_docs.append(sent2)

        #類似度を求め、最も高いものを出力
        model = Doc2Vec(documents=training_docs)
        similar = model.docvecs.most_similar(news, topn=1)
        
        print(similar)

### cos類似度を用いて類似ニュースを算出

In [47]:
similar(document, news_num=1, sim='cos')


cos類似度　:　0.0465
シックな店内で絶品軍鶏を食べくらべ粋な坂の街・神楽坂。この街には、通好みの飲食店が少なからずあり、「匠軍鶏郭」もその内の一軒。日本三大軍鶏の東京軍鶏、薩摩軍鶏、奥久慈しゃもをはじめ、青森軍鶏ロックに信州黄金軍鶏など、厳選した軍鶏を、炭火焼きや鍋などシンプルなスタイルで楽しむメニューは、どれも滋味豊かなものばかり。特に炭火焼きでいただく三大軍鶏盛り合わせ（2400円）は、昔ながらの歯ごたえのある奥久慈しゃもと、幻の味、東京軍鶏、そして、やわらかく甘みも堪能できる薩摩軍鶏のそれぞれの肉質を味わえる逸品として、軍鶏通はもちろん、初心者でも軍鶏を存分に楽しめると評判です。そんな極上の軍鶏をさらに引き立ててくれるお酒も名酒が揃い踏み。フランスのロワール、ローヌ、アルザス、ボージョレのビオワインや、ふくよかな米の甘みが喉越しを潤してくれる純米酒「澤屋まつもと」など、料理との相性を考えて、もてなされる至福のマリアージュは、思わず笑みを浮かべてしまうほど、幸せな時間を与えてくれます。各席をパテーションで区切った、プライベート空間重視の1階のテーブル席に加え、10人から40人まで使用できる、掘りごたつが置かれた宴会用の地下1階と、それぞれのスタイルやニーズに合わせて空間が用意されているのも魅力です。神楽坂の地元客やツウが通う、粋な店。ちょっと足をのばして、本物の軍鶏の味を満喫してみてはいかがですか？5000円のお食事券を2名様にプレゼント匠軍鶏郭応募はこちらから！（※応募にはlivedoorIDが必要となります）匠軍鶏郭東京都新宿区揚場町2-1軽子坂MNビル1F軍鶏郭-公式サイト


### 別の検出手法としてdoc2vecを用いて類似ニュースを算出

In [46]:
#選択したニュースと似ているニュースを探す(文章、類似度の順でアウトプットする)
similar(document, news_num=10, sim='doc2vec')


[('カセットHDDで録画番組を管理！ネット対応のHDDレコーダーが凄いテレビが好きな人の中には、趣味や仕事で役に立つテレビ番組をテレビに内蔵のハードディスク（HDD）や外付けのHDDに録画している人がいるだろう。そうした録画番組は、著作権保護の関係上、録画したテレビでしか見ることができない。HDDレコーダーなら、録画番組を見たいテレビに繋ぎかえればよいわけだが、手間が掛かるうえに、そこまでしてやりたくはないだろう。ましてや、テレビに内蔵のHDDであれば、ほかのテレビには繋げられないので、お手上げだ。とはいえ、録画番組はいつでもどこでも見たいというのが正直なところだろう。実は、そんな密かな願いをかなえてくれる魔法のようなアイテムがある。それがアイ・オー・データ機器のDTCP-IP対応ハイビジョンレコーディングハードディスク「RECBOX」だ。まずは、zigsowのレビューの中から、RECBOX「HVL-AVシリーズ」の使い勝手を紹介したのち、最新モデルとなる「HVL-AVS」がタダで貰えるスペシャルな企画を紹介しよう。※本記事は、zigsow向けに執筆された原稿に手を加えずにそのまま掲載している。これはPCコントロールができる「家電」です。今回は、I・ODATA様の「DTCP-IP対応ハイビジョンレコーディングハードディスクRECBOX（HVL-AVシリーズ）」のプレミアムレビューのレビューアーに選出していただき、ありがとうございます。「DTCP-IP対応ハイビジョンレコーディングハードディスク」なるものがどういう働きで、一般的な外部接続ハードディスクとどう違うのか、をレポートします。設置と使用に分けて二部構成でお送りします。まずレビュー対象品の到着から．．．外箱でかっっっっっっっっっっ大きな箱をあけると半分ほどの高さまでしか入っていない．．．中箱を開けると外箱の大きさでビビリましたが、中身は意外にコンパクト！中身は説明書類（クイックリファレンス2葉＋説明書）と以下の4点。本体とACアダプタ、電源線、LANケーブル説明書もあまり詳しくなくちょっと？？クイックリファレンス（のうち1枚）はREGZAとRECBOXの直結にフォーカスした内容で（あと1枚はスカパー！関係）、家庭内LANが既にある環境は説明書を見る。「ネットワークに接続する」というところが該当なのですが、全

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


音声認識機能を用いた英会話学習とそのシステムについてのニュースとの類似度を判定した。

＜cos類似度を用いた検出＞
商品紹介やサービス紹介を行っている点、地域の話題が出ている点などが類似しているがcos類似度スコアは非常に低いためあまり似ているとは思えない。

＜doc2vecを用いた検出＞
テクノロジー系の商品紹介行っている点が類似しており、また仕事で活かすことのできる製品である点も一致している。また商品紹介においても、機能の紹介など文章の特徴に類似性
が見られる。


# 【問題6】感情分析
目的  
NLP定番の感情分析の経験  
英語の処理の実践  

In [48]:
# IMDBをカレントフォルダにダウンロード
#!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# 解凍
#!tar zxf aclImdb_v1.tar.gz
# aclImdb/train/unsupはラベル無しのため削除
#!rm -rf aclImdb/train/unsup
# IMDBデータセットの説明を表示
#!cat aclImdb/README

--2019-07-30 15:50:28--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  37.2MB/s    in 2.2s    

2019-07-30 15:50:30 (37.2 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an a

In [0]:
# サブフォルダまで自動で読み込んでもらう
from sklearn.datasets import load_files

train_review = load_files('./aclImdb/train/', encoding='utf-8')
train_text, train_y = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
test_text, test_y = test_review.data, test_review.target

# 【問】
IMDBという映画に対するレビューのデータセットを使います。
良いレビューか悪いレビューかを判定するモデルを作ってください。
テストデータに対する正解率が90%を超えるまで、調査=>実行=>改善を繰り返してください。
前処理になぜその処理をしたのかを書くとエンジニアリングとしても完璧です。
注意: 必ず間違っていたデータを観察してください。

### データの読み込み

In [0]:
train_review = load_files('./aclImdb/train/', encoding='utf-8')
X_train, y_train = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
X_test, y_test = test_review.data, test_review.target

In [100]:
#学習用データの確認
print(X_train[0])
print(y_train[0])

Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty.
1


### 学習用とテスト用のデータを合わせてテキストクリー二ングを実施

In [101]:
#学習用とテスト用のデータを合わせてテキストクリー二ング
X_train[len(X_train): len(X_train)] = X_test

#<br>
br = '<.*?>'

#特殊文字
tokusyu = "[\?!\/,.'\(\):\"-;]"

#連結
reg_str = f'{br}|{tokusyu}'

#テキストクリーニングしてリストに戻す
text = []

for i in X_train:
    #ぜんぶ小文字に
    i = i.lower()
    
    #正規表現で削除
    remove = re.compile(reg_str)
    text.append(re.sub(remove, ' ', i))

#レビューの確認
print(text[0])

zero day leads you to think  even re think why two boys young men would do what they did   commit mutual suicide via slaughtering their classmates  it captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own mutual world via coupled destruction   it is not a perfect movie but given what money time the filmmaker and actors had   it is a remarkable product  in terms of explaining the motives and actions of the two young suicide murderers it is better than  elephant    in terms of being a film that gets under our  rationalistic  skin it is a far  far better film than almost anything you are likely to see    flawed but honest with a terrible honesty 


In [0]:
#学習用とテスト用に戻す
X_train = text[:25000]
X_test = text[25000:]

In [0]:
NGRAM_RANGE = (1, 2)
TOP_K = 10000
TOKEN_MODE = 'word'
MIN_DOC_FREQ = 2

def ngram_vectorize(train_texts, train_labels, test_texts):
    kwargs = {
        'ngram_range' : NGRAM_RANGE,
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : TOKEN_MODE,
        'min_df' : MIN_DOC_FREQ,
    }
    
    # Learn Vocab from train texts and vectorize train and val sets
    tfidf_vectorizer = TfidfVectorizer(**kwargs)
    X_train = tfidf_vectorizer.fit_transform(train_texts)
    X_test = tfidf_vectorizer.transform(test_texts)
    
    # Select best k features, with feature importance measured by f_classif
    selector = SelectKBest(f_classif, k=min(TOP_K, X_train.shape[1]))
    selector.fit(X_train, train_labels)
    X_train = selector.transform(X_train).astype('float32')
    X_test = selector.transform(X_test).astype('float32')
    return X_train, X_test

In [104]:
X_train, X_test = ngram_vectorize(X_train, y_train, X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


In [105]:
# 入力サイズは映画レビューで使われている語彙数
vocab_size = X_train.shape[1:]

#モデル作成
model = Sequential()
model.add(Dropout(0.5, input_shape=vocab_size, ))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_13 (Dropout)         (None, 10000)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout_14 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_15 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [0]:
#コンパイル
model.compile(optimizer=Adam(lr=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

#コールバック
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]

In [0]:
#学習データと検証データに分割
(X_train, X_val, y_train, y_val) = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

In [109]:
#学習
history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=128,
                    validation_data=(X_val, y_val), 
                    verbose=1,
                    callbacks=callbacks)

Train on 17500 samples, validate on 7500 samples
Epoch 1/10
17500/17500 [==============================] - 2s 92us/step - loss: 0.6559 - acc: 0.6534 - val_loss: 0.5441 - val_acc: 0.8715
Epoch 2/10
17500/17500 [==============================] - 1s 56us/step - loss: 0.4642 - acc: 0.8231 - val_loss: 0.3160 - val_acc: 0.8985
Epoch 3/10
17500/17500 [==============================] - 1s 55us/step - loss: 0.3424 - acc: 0.8769 - val_loss: 0.2482 - val_acc: 0.9065
Epoch 4/10
17500/17500 [==============================] - 1s 56us/step - loss: 0.2979 - acc: 0.8893 - val_loss: 0.2243 - val_acc: 0.9136
Epoch 5/10
17500/17500 [==============================] - 1s 54us/step - loss: 0.2765 - acc: 0.8995 - val_loss: 0.2143 - val_acc: 0.9180
Epoch 6/10
17500/17500 [==============================] - 1s 55us/step - loss: 0.2604 - acc: 0.9030 - val_loss: 0.2085 - val_acc: 0.9185
Epoch 7/10
17500/17500 [==============================] - 1s 54us/step - loss: 0.2420 - acc: 0.9109 - val_loss: 0.2074 - val_acc:

In [110]:
#結果表示
history = history.history
test_acc = history['val_acc'][-1]
test_loss = history['val_loss'][-1]

print('loss :', test_loss)
print('acc :', test_acc)

loss : 0.19937214992841085
acc : 0.9213333333651225


In [111]:
np.where((np.where(model.predict(X_test) < 0.5, 0, 1).reshape(-1) == test_y) == False)[0][:5]

array([ 0, 11, 25, 32, 37])

In [112]:
#間違ったテキストを表示
#１つ目
print(test_x[0])
print(test_y[0])

#2つ目
print(test_x[11])
print(test_y[11])


#3つ目
print(test_x[25])
print(test_y[25])

#4つ目
print(test_x[32])
print(test_y[32])


#5つ目
print(test_x[37])
print(test_y[37])

Don't hate Heather Graham because she's beautiful, hate her because she's fun to watch in this movie. Like the hip clothing and funky surroundings, the actors in this flick work well together. Casey Affleck is hysterical and Heather Graham literally lights up the screen. The minor characters - Goran Visnjic {sigh} and Patricia Velazquez are as TALENTED as they are gorgeous. Congratulations Miramax & Director Lisa Krueger!
1
Even Disney are guilty of the cash cow disease, after the roaring success of The Love Bug in 1968, the house of mouse cashed in with Herbie Rides Again, Herbie Goes To Monte Carlo, and Herbie Goes Bananas. Neither sequel capturing the charm and inoffensive appeal of The Love Bug back in 68, in this one we find race driver Jim Douglas and his sidekick Wheely Applegate, entering Herbie in the Monte Carlo Rally. Naturally things outside of the race start to take over priorities, they get mixed up in a diamond robbery and Herbie falls in love with another car!. The car 

hateという単語が二回繰り返されたレビューでは謝ってネガティブ評価に予測したり、比較的良い評価を一部加えながら１０点中４点と辛口評価されたレビューではポジティブに予測したりしていた。単語それぞれのポジティブ性、ネガティブ性にかなり影響を受けている可能性がある。